# Testons les différents modèles

In [21]:
#!pip install "neptune-client<0.9.0" neptune-contrib

     ---------------------------------------- 0.0/205.3 kB ? eta -:--:--
     -------------------------------------- 205.3/205.3 kB 6.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached neptune-contrib-0.28.1.tar.gz (74 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for neptune-client: started
  Running setup.py install for neptune-client: finished with status 'done'
  Running setup.py install for neptune-contrib: started
  Running setup.py install for neptune-contrib: finished with status 'done'


  DEPRECATION: neptune-client is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  DEPRECATION: neptune-contrib is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [22]:
import os
from pathlib import Path
import math
import json
import numpy as np
import torch
import matplotlib.pyplot as plt
from models import get_model
from toolbox.losses import triplet_loss
from toolbox import metrics
from loaders.data_generator import QAP_Generator
from loaders.siamese_loaders import siamese_loader
from toolbox.metrics import all_losses_acc, accuracy_linear_assignment
from toolbox.utils import check_dir

In [23]:
def get_device_config(model_path):
    config_file = os.path.join(model_path,'config.json')
    with open(config_file) as json_file:
        config_model = json.load(json_file)
    use_cuda = not config_model['cpu'] and torch.cuda.is_available()
    device = 'cuda' if use_cuda else 'cpu'
    return config_model, device

def load_model(model_path, config, device):
    model = get_model(config['arch'])
    model.to(device)
    model_file = os.path.join(model_path,'model_best.pth.tar')
    if device == 'cpu':
        checkpoint = torch.load(model_file,map_location=torch.device('cpu'))
    else:
        checkpoint = torch.load(model_file)
    model.load_state_dict(checkpoint['state_dict'])
    return model

def creat_args(config, num_ex = 1000):
    args = config['data']
    args['num_examples_test'] = num_ex
    n_vertices = args['n_vertices']
    edge_density = args['edge_density']
    deg = (n_vertices)*edge_density
    print(f'graphs with {n_vertices} vertices and average degree {deg}')
    return args, deg

def acc_2_error(mean_acc, q_acc):
    error = q_acc-mean_acc[:,np.newaxis]
    error[:,0] = -error[:,0]
    return error

def compute_all(list_noise,args,path_dataset,model,bs=50):
    num_batches = math.ceil(args['num_examples_test']/bs)
    all_losses = np.zeros((len(list_noise),num_batches))
    all_acc = np.zeros((len(list_noise),args['num_examples_test']))
    for i,noise in enumerate(list_noise):
        args['noise'] = noise
        gene_test = QAP_Generator('test', args, path_dataset)
        gene_test.load_dataset()
        test_loader = siamese_loader(gene_test, bs, gene_test.constant_n_vertices)
        all_losses[i,:], all_acc[i,:] = all_losses_acc(test_loader,model,criterion,device,eval_score=accuracy_linear_assignment)
    return all_losses, all_acc

def compute_quant(all_acc,quant_low=0.1,quant_up=0.9):
    mean_acc = np.mean(all_acc,1)
    num = len(mean_acc)
    q_acc = np.zeros((num,2))
    for i in range(num):
        q_acc[i,:] = np.quantile(all_acc[i,:],[quant_up, quant_low])
    return mean_acc, q_acc

In [ ]:
!python commander.py train with arch.arch_gnn="mgnn"

Heading to Training.
Current problem :  qap
Using device: cpu
Models saved in  C:\Users\ordinateur\experiments-gnn/qap/qap_res\mgnn_4_ErdosRenyi_200_0.0015\03-01-26-21-16
Reading dataset at C:\Users\ordinateur\experiments-gnn/qap/data/QAP_ErdosRenyi_ErdosRenyi_20000_200_1.0_0.1_0.0015\train.pkl


WARNING - commander - No observers have been added to this run
INFO - commander - Running command 'train'
INFO - commander - Started
ERROR - commander - Failed after 0:00:17!
Traceback (most recent calls WITHOUT Sacred internals):
  File "c:\Users\ordinateur\Documents\GitHub\Project_deep_learning\commander.py", line 131, in train
    gene_train.load_dataset(use_dgl)
  File "c:\Users\ordinateur\Documents\GitHub\Project_deep_learning\loaders\data_generator.py", line 213, in load_dataset
    data = torch.load(path)
  File "c:\Users\ordinateur\Documents\GitHub\Project_deep_learning\.venv\lib\site-packages\torch\serialization.py", line 809, in load
    return _load(opened_zipfile, map_location, pickle_module, **pickle_load_args)
  File "c:\Users\ordinateur\Documents\GitHub\Project_deep_learning\.venv\lib\site-packages\torch\serialization.py", line 1172, in _load
    result = unpickler.load()
  File "c:\Users\ordinateur\Documents\GitHub\Project_deep_learning\.venv\lib\site-packages\torch\ser

Heading to Training.
Current problem :  qap
Using device: cpu
Models saved in  C:\Users\ordinateur\experiments-gnn/qap/qap_res\fgnn_4_ErdosRenyi_200_0.0015\03-01-26-19-21
Creating dataset at C:\Users\ordinateur\experiments-gnn/qap/data/QAP_ErdosRenyi_ErdosRenyi_20000_200_1.0_0.1_0.0015\train.pkl
Saving dataset at C:\Users\ordinateur\experiments-gnn/qap/data/QAP_ErdosRenyi_ErdosRenyi_20000_200_1.0_0.1_0.0015\train.pkl
Creating dataset at C:\Users\ordinateur\experiments-gnn/qap/data/QAP_ErdosRenyi_ErdosRenyi_20000_200_1.0_0.1_0.0015\train_dgl.pkl
Converting data to DGL format
Conversion ended.
Saving dataset at C:\Users\ordinateur\experiments-gnn/qap/data/QAP_ErdosRenyi_ErdosRenyi_20000_200_1.0_0.1_0.0015\train_dgl.pkl
Creating dataset at C:\Users\ordinateur\experiments-gnn/qap/data/QAP_ErdosRenyi_ErdosRenyi_1000_200_1.0_0.1_0.0015\val.pkl
Saving dataset at C:\Users\ordinateur\experiments-gnn/qap/data/QAP_ErdosRenyi_ErdosRenyi_1000_200_1.0_0.1_0.0015\val.pkl
Creating dataset at C:\Users\

WARNING - commander - No observers have been added to this run
INFO - commander - Running command 'train'
INFO - commander - Started

100%|██████████| 20000/20000 [11:06<00:00, 30.02it/s]

100%|██████████| 20000/20000 [03:25<00:00, 97.35it/s] 

100%|██████████| 1000/1000 [00:42<00:00, 23.62it/s]

100%|██████████| 1000/1000 [00:08<00:00, 118.52it/s]
ERROR - commander - Failed after 1:54:59!
Traceback (most recent calls WITHOUT Sacred internals):
  File "c:\Users\ordinateur\Documents\GitHub\Project_deep_learning\commander.py", line 159, in train
    trainer.train_triplet(train_loader,model,optimizer,exp_helper,device,epoch,eval_score=True,print_freq=train['print_freq'])
  File "c:\Users\ordinateur\Documents\GitHub\Project_deep_learning\trainer.py", line 27, in train_triplet
    raw_scores = model(data)
  File "c:\Users\ordinateur\Documents\GitHub\Project_deep_learning\.venv\lib\site-packages\torch\nn\modules\module.py", line 1501, in _call_impl
    return forward_call(*args, **kwargs)
  